In [3]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `C:\Users\tge13\Documents\optimal_control_julia\lec_5\Project.toml`


In [9]:
Pkg.add("Pluto")

   Resolving package versions...
   Installed RegistryInstances ─────────── v0.1.0
   Installed ExpressionExplorer ────────── v1.0.0
   Installed PlutoDependencyExplorer ───── v1.0.4
   Installed PrecompileSignatures ──────── v3.0.3
   Installed Malt ──────────────────────── v1.1.1
   Installed Tables ────────────────────── v1.11.1
   Installed ExproniconLite ────────────── v0.10.6
   Installed FuzzyCompletions ──────────── v0.5.3
   Installed TableTraits ───────────────── v1.0.1
   Installed IteratorInterfaceExtensions ─ v1.0.0
   Installed Tricks ────────────────────── v0.1.8
   Installed HypertextLiteral ──────────── v0.9.5
   Installed MIMEs ─────────────────────── v0.1.4
   Installed Configurations ────────────── v0.17.6
   Installed DataValueInterfaces ───────── v1.0.0
   Installed OpenSSL_jll ───────────────── v3.0.13+0
   Installed MsgPack ───────────────────── v1.2.1
   Installed LazilyInitializedFields ───── v1.2.2
   Installed Pluto ─────────────────────── v0.19.40
    Updat

In [8]:
using LinearAlgebra
using ForwardDiff
using PyPlot

[ Info: Precompiling PyPlot [d330b81b-6aea-500a-939a-2ce795aea3ee]
[ Info: Installing matplotlib via the Conda matplotlib package...
[ Info: Running `conda install -y matplotlib` in root environment


Channels:
 - conda-forge
Platform: win-64
Solving environment: ...working... done




==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.1.2

Please update conda by running

    $ conda update -n base -c conda-forge conda





## Package Plan ##

  environment location: C:\Users\tge13\.julia\conda\3\x86_64

  added / updated specs:
    - matplotlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    brotli-1.1.0               |       hcfcfb64_1          19 KB  conda-forge
    brotli-bin-1.1.0           |       hcfcfb64_1          20 KB  conda-forge
    contourpy-1.2.0            |  py310h232114e_0         186 KB  conda-forge
    cycler-0.12.1              |     pyhd8ed1ab_0          13 KB  conda-forge
    fonttools-4.49.0           |  py310h8d17308_0         1.8 MB  conda-forge
    freetype-2.12.1            |       hdaf720e_2         498 KB  conda-forge
    gettext-0.21.1             |       h5728263_0         5.3 MB  conda-forge
    glib-2.78.4                |       h12be248_4         495 KB  conda-forge
    glib-tools-2.78.4          |       h12be248_4         142 KB  conda-forge
    gst-plugins-base-1.22.9

In [ ]:
Q = Diagonal([0.5; 1])
function f(x)
    return 0.5*(x-[1; 0])'*Q*(x-[1; 0])
end
function ∇f(x)
    return Q*(x-[1; 0])
end
function ∇2f(x)
    return Q
end

In [ ]:
A = [1.0 -1.0]
b = -1.0
function c(x)
    return dot(A,x) - b
end
function ∂c(x)
    return A
end

In [ ]:
function plot_landscape()
    Nsamp = 20
    Xsamp = kron(ones(Nsamp),LinRange(-4,4,Nsamp)')
    Ysamp = kron(ones(Nsamp)',LinRange(-4,4,Nsamp))
    Zsamp = zeros(Nsamp,Nsamp)
    for j = 1:Nsamp
        for k = 1:Nsamp
            Zsamp[j,k] = f([Xsamp[j,k]; Ysamp[j,k]])
        end
    end
    contour(Xsamp,Ysamp,Zsamp)

    xc = LinRange(-4,3,Nsamp)
    plot(xc,xc.+1,"y")
end

plot_landscape()

In [ ]:
function La(x,λ,ρ)
    p = max(0,c(x))
    return f(x) + λ*p + (ρ/2)*(p'*p)
end

In [ ]:
function newton_solve(x0,λ,ρ)
    x = x0
    p = max.(0,c(x))
    C = zeros(1,2)
    if c(x) ≥ 0
        C = ∂c(x)
    end
    g = ∇f(x) + (λ+ρ*p)*C'
    while norm(g) ≥ 1e-8
        H = ∇2f(x) + ρ*C'*C
        Δx = -H\g
        
        x = x+Δx
        
        p = max.(0,c(x))
        C = zeros(1,2)
        if c(x) ≥ 0
            C = ∂c(x)
        end
        g = ∇f(x) + (λ+ρ*p)*C'
    end
    
    return x
end

In [ ]:
xguess = [-3; 2]
λguess = [0.0]
ρ = 1.0
plot_landscape()
plot(xguess[1], xguess[2], "rx")

In [ ]:
xnew = newton_solve(xguess[:,end],λguess[end],ρ)
λnew = max.(0,λguess[end]+ρ*c(xnew))
xguess = [xguess xnew]
λguess = [λguess λnew]
ρ = 10*ρ
plot_landscape()
plot(xguess[1,:], xguess[2,:], "rx")